In [ ]:
import math
import numpy as np
from scipy import signal
import pandas as pd
import altair as alt

In [ ]:
df_L = pd.read_csv(
    filepath_or_buffer='../datas/inroom/m001/L.txt', 
    sep=',', 
    header=14,
    names=['Freq', 'dB', 'Phase'])
df_R = pd.read_csv(
    filepath_or_buffer='../datas/inroom/m001/R.txt', 
    sep=',', 
    header=14,
    names=['Freq', 'dB', 'Phase'])

In [ ]:
def graph_freq_axis(max_freq):
    return alt.X(
            "Freq:Q",
            title="Freqency (Hz)",
            scale=alt.Scale(type="log", base=10, nice=False, domain=[20, max_freq]),
            axis=alt.Axis(format="s"))

def graph_base(df, max_freq, color):
    chart_base = alt.Chart(df.loc[df.Freq<max_freq]).mark_line().encode(
        graph_freq_axis(max_freq),
        color=alt.value(color)
    )
    return chart_base.encode(y=alt.Y('dB:Q', scale=alt.Scale(domain=[30, 80]), axis=alt.Axis(grid=True)))

def graph_loess(df, max_freq, color):
    chart = graph_base(df, max_freq, color)
    return chart, chart.transform_loess('Freq', 'dB', bandwidth=0.16).mark_line().encode(color=alt.value('red'))

def graph_phase(df, max_freq, color):
    return alt.Chart(df.loc[df.Freq<max_freq]).mark_line().encode(
        graph_freq_axis(max_freq),
        y=alt.Y('Phase:Q', 
                scale=alt.Scale(zero=False, domain=[-180, 180+360], nice=False),
                axis=alt.Axis(
                    tickCount=36, 
                    tickMinStep=30,
                    labelExpr="datum.value % 60 | datum.value > 180 ? null : datum.label", 
                        #format=f'format(datum.Phase, "3d")')),
                    format=">-3d")),
        color=alt.value(color)
    )

def graph_groupdelay(df, max_freq, color):
    return alt.Chart(df.loc[df.Freq<max_freq]).mark_line().encode(
        graph_freq_axis(max_freq),
        y=alt.Y('GroupDelay:Q', 
                scale=alt.Scale(zero=False, nice=True)),
        color=alt.value(color)
    )

def graph(df, df_target, max_freq, title):
    chart_freq, chart_freq_loess = graph_loess(df, max_freq, 'blue')
    chart_phase = graph_phase(df, max_freq, 'green')
    chart_target = graph_base(df_target, max_freq, 'black')
    return alt.layer(chart_freq + chart_freq_loess + chart_target, chart_phase).resolve_scale(y='independent').properties(title=title)

    

In [ ]:
target = [(20, 46), (35, 64), (50, 66), (200, 68), (300, 68), (1000, 67), (19999, 60)]
target_x = np.concatenate([np.logspace(math.log10(target[i][0]), math.log10(target[i+1][0]), 10) for i in range(0, len(target)-1)])
target_y = np.concatenate([np.linspace(target[i][1], target[i+1][1], 10) for i in range(0, len(target)-1)])

In [ ]:
df_target = pd.DataFrame({'Freq': target_x, 'dB': target_y})

In [ ]:
max_freq = 3000
chart_L = graph(df_L, df_target, max_freq, 'Left')
chart_R = graph(df_R, df_target, max_freq, 'Right')

In [ ]:
chart_L | chart_R

In [ ]:
unwrapped = np.unwrap(df_L.Phase.values)
group_delay = np.gradient(unwrapped)

In [ ]:
freq = df_L.dB.values
phase = df_L.Phase.values

In [ ]:
unwrapped = []
unwrapped.append(phase[0])
group_delay = []
group_delay.append((phase[1]-phase[0])/(freq[1]-freq[0]))
n = 0
for i in range(1, len(phase)):
    if abs(phase[i]-phase[i-1])>300:
        n = n+1
    up = phase[i] - n*360
    unwrapped.append(up)
    der = (unwrapped[-1]-unwrapped[-2])/(freq[i]-freq[i-1])
    if der>500:
        der = 500
    if der < -500:
        der = 500
    group_delay.append(der)

In [ ]:
len(phase), len(freq), len(unwrapped), len(group_delay)

In [ ]:
df_L['GroupDelay'] = group_delay

In [ ]:
graph_groupdelay(df_L, 300, 'red')